In [29]:
import pandas as pd
import numpy as np
from pathlib import Path

In [30]:
rename_map = {
    "IMBA_5_Beta_FeatureWise": "Ours (Beta, FW)",
    "IMBA_5_Exp_FeatureWise": "Ours (Exp, FW)",
    "GreedySemanticEnrichedMTEAveDissimilarity": "CoCoA_NMTE",
    "GreedySemanticEnrichedMaxprobAveDissimilarity": "CoCoA_MSP",
    "GreedySemanticEnrichedPPLAveDissimilarity": "CoCoA_PPL",
    "MonteCarloSequenceEntropy": "MCSE",
    "MonteCarloNormalizedSequenceEntropy": "MCNSE",
    "SemanticEntropy": "Semantic_Entropy",
    "SAR_t0.001": "SAR",
    "DegMat_NLI_score_entail": "DegMat",
    "EigValLaplacian_NLI_score_entail": "EigValLaplacian",
    "MaximumSequenceProbability": "MSP",
    "Perplexity": "PPL",
    "MeanTokenEntropy": "NMTE",
}

In [31]:
dataset_col_rename = {
    "xsum": "XSum",
    "wmt14_fren": "WMT14FrEn",
    "wmt19_deen": "WMT19DeEn",
    "coqa_no_context": "CoQA",
    "trivia": "Trivia",
    "mmlu": "MMLU",
    "gsm8k_cot": "GSM8k",
}

def rename_dataset_columns(df):
    # Only rename columns that match keys, ignore 'Method' and 'mean'
    new_cols = {}
    for c in df.columns:
        if c in dataset_col_rename:
            new_cols[c] = dataset_col_rename[c]
    return df.rename(columns=new_cols)

In [32]:
# Load data
paths = {
    "LLaMA-8B": "../resources/llm_resources/llama8b_results.csv",
    "Mistral-7B": "../resources/llm_resources/mistral7b_results.csv",
    "Falcon-7B": "../resources/llm_resources/falcon7b_results.csv",
}
dfs = {name: pd.read_csv(path) for name, path in paths.items()}

# Apply dataset column renaming to all dataframes
dfs = {name: rename_dataset_columns(df) for name, df in dfs.items()}

# Selection config
FOUR_METHODS_ORIG = [
    "IMBA_5_Beta_FeatureWise",
    "IMBA_5_Exp_FeatureWise",
    "Master_1_Exp_FeatureWise",
    "Master_1_Beta_FeatureWise",
]
EXCLUDE_PREFIXES = ("IMBA", "Master", "CoCoA")

In [33]:
def filter_methods(df):
    m = df["Method"].astype(str)
    keep_mask = m.isin(FOUR_METHODS_ORIG) | ~m.str.startswith(EXCLUDE_PREFIXES)
    return df.loc[keep_mask].copy()

def dataset_columns(df):
    # dataset columns = everything except 'Method' and 'mean'
    return [c for c in df.columns if c not in ("Method", "mean")]

In [34]:
out_dir = Path("../")
out_dir.mkdir(parents=True, exist_ok=True)

# --- Per-model LaTeX tables ---
for model_name, df in dfs.items():
    dff = filter_methods(df)

    # Order rows: four methods first (in given order), then the rest alphabetically by RENAMED name
    is_four = dff["Method"].isin(FOUR_METHODS_ORIG)
    dff_four = dff[is_four].copy()
    dff_rest = dff[~is_four].copy()

    # Apply rename for output
    dff_four["Method"] = dff_four["Method"].replace(rename_map)
    dff_rest["Method"] = dff_rest["Method"].replace(rename_map)

    # Preserve the order of the four, but based on their renamed labels
    order_map = {
        rename_map.get("IMBA_5_Beta_FeatureWise", "IMBA_5_Beta_FeatureWise"): 0,
        rename_map.get("IMBA_5_Exp_FeatureWise", "IMBA_5_Exp_FeatureWise"): 1,
        rename_map.get("Master_1_Exp_FeatureWise", "Master_1_Exp_FeatureWise"): 2,
        rename_map.get("Master_1_Beta_FeatureWise", "Master_1_Beta_FeatureWise"): 3,
    }
    dff_four["__order"] = dff_four["Method"].map(order_map)

    dff_rest = dff_rest.sort_values("Method")
    dff_rest["__order"] = 1000 + np.arange(len(dff_rest))

    dff_out = (
        pd.concat([dff_four, dff_rest], ignore_index=True)
        .sort_values("__order")
        .drop(columns="__order")
    )

    # Round numbers for LaTeX display
    dff_out_fmt = dff_out.copy()
    for col in dff_out_fmt.columns:
        if col != "Method":
            dff_out_fmt[col] = dff_out_fmt[col].map(lambda x: f"{x:.3f}")

    caption = f"{model_name}: filtered methods (renamed). Higher is better."
    label = f"tab:{model_name.lower().replace('-', '').replace(' ', '')}_filtered_renamed"
    latex = dff_out_fmt.to_latex(index=False, caption=caption, label=label, escape=False)
    (out_dir / f"{model_name}_filtered_renamed.tex").write_text(latex)

In [35]:

# --- Joint table: choose top-3 datasets by average performance of the four methods across models ---

# 1) Intersection of dataset columns
common_cols = None
for df in dfs.values():
    cols = set(dataset_columns(df))
    common_cols = cols if common_cols is None else (common_cols & cols)
common_cols = sorted(list(common_cols))

In [36]:
common_cols

['CoQA', 'GSM8k', 'MMLU', 'Trivia', 'WMT14FrEn', 'WMT19DeEn', 'XSum']

In [37]:
# 2) Average across models & methods
avg_scores = {}
for col in common_cols:
    vals = []
    for df in dfs.values():
        # sub = df[df["Method"].isin(FOUR_METHODS)]
        sub = df
        if col in sub.columns and not sub.empty:
            vals.extend(sub[col].tolist())
    if vals:
        avg_scores[col] = np.mean(vals)

In [38]:
# 3) Pick top-3 datasets
top3 = sorted(avg_scores, key=avg_scores.get, reverse=True)[:3]

rows = []
for orig_method in dff["Method"].unique():
    row = {"Method": rename_map.get(orig_method, orig_method)}
    for model_name, df in dfs.items():
        sub = df[df["Method"] == orig_method]
        for col in top3:
            key = f"{model_name} {col}"
            row[key] = float(sub.iloc[0][col]) if not sub.empty else np.nan
    rows.append(row)
joint_df = pd.DataFrame(rows)

In [39]:
joint_df['mean'] = joint_df.iloc[:, 1:].mean(1)

In [40]:
joint_df_sorted = joint_df.sort_values(by="mean", ascending=False)

In [41]:
joint_df_sorted.loc['mean'] = joint_df_sorted.iloc[:, 1:].mean(0)

In [42]:
# Format
joint_fmt = joint_df_sorted.copy()
for col in joint_fmt.columns:
    if col != "Method":
        joint_fmt[col] = joint_fmt[col].map(lambda x: "" if pd.isna(x) else f"{x:.3f}")

joint_caption = (
    "Top-3 datasets by average performance of the four selected methods (renamed) across models. "
    "Rows: methods; columns: (model, dataset). Higher is better."
)
joint_label = "tab:joint_top3_renamed"
(out_dir / "joint_top3_renamed.tex").write_text(
    joint_fmt.to_latex(index=False, caption=joint_caption, label=joint_label, escape=False)
)

sorted([p.name for p in out_dir.glob("*.tex")])

['Falcon-7B_filtered_renamed.tex',
 'LLaMA-8B_filtered_renamed.tex',
 'Mistral-7B_filtered_renamed.tex',
 'joint_top3_renamed.tex']